# 실시간 제스처 인식으로 컨트롤하는 AI 비서

- MediaPipe는 Google에서 개발한 크로스 플랫폼 실시간 ML 기반 영상 처리 프레임워크이다. 특히 웹캠, 카메라, 영상 스트림에서 얼굴, 손, 포즈, 객체 등을 인식하는 데 강력하게 사용된다.

In [1]:
# ! pip install mediapipe

In [1]:
from flask import Flask, render_template, Response
import cv2
from gesture_detector import GestureDetector
from openai_client import get_ai_response

app = Flask(__name__)

cap = cv2.VideoCapture(1)

detector = GestureDetector()

latest_fingers = 0
latest_response = ""

def gen_frames():
    global latest_fingers, latest_response
    while True:
        success, frame = cap.read()
        if not success:
            break

        fingers = detector.count_fingers(frame)
        if fingers != latest_fingers and fingers > 0:
            latest_fingers = fingers
            latest_response = get_ai_response(fingers)

        cv2.putText(frame, f'Fingers: {latest_fingers}', (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, latest_response, (10, 70),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)

        _, buffer = cv2.imencode('.jpg', frame)
        frame = buffer.tobytes()
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/video_feed')
def video_feed():
    return Response(gen_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    app.run(debug=False)

# 한글 지원

In [ ]:
# !pip install mediapipe pillow

In [ ]:
from flask import Flask, render_template, Response
import cv2
from gesture_detector import GestureDetector
from openai_client import get_ai_response
from PIL import ImageFont, ImageDraw, Image
import numpy as np
import os

app = Flask(__name__)
cap = cv2.VideoCapture(1)
detector = GestureDetector()

latest_fingers = 0
latest_response = ""

# 한글 텍스트를 프레임에 표시하는 함수
def draw_korean_text(frame, text, position=(10, 30), font_size=24, font_path=None):
    if font_path is None:
        font_path = "C:/Windows/Fonts/malgun.ttf"  # Windows 기준
        # Linux일 경우 예: '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'

    image_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(image_pil)
    font = ImageFont.truetype(font_path, font_size)
    draw.text(position, text, font=font, fill=(255, 255, 255, 255))
    return cv2.cvtColor(np.array(image_pil), cv2.COLOR_RGB2BGR)

def gen_frames():
    global latest_fingers, latest_response
    while True:
        success, frame = cap.read()
        if not success:
            break

        fingers = detector.count_fingers(frame)
        if fingers != latest_fingers and fingers > 0:
            latest_fingers = fingers
            latest_response = get_ai_response(fingers)

        # 한글 텍스트 표시
        frame = draw_korean_text(frame, f'손가락 수: {latest_fingers}', (10, 30), font_size=28)
        frame = draw_korean_text(frame, latest_response, (10, 70), font_size=24)

        _, buffer = cv2.imencode('.jpg', frame)
        frame = buffer.tobytes()
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/video_feed')
def video_feed():
    return Response(gen_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [22/Apr/2025 23:53:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2025 23:53:32] "GET /video_feed HTTP/1.1" 200 -


### AJAX를 통해 서버와 실시간 상호작용

In [1]:
from flask import Flask, render_template, Response
import cv2
from gesture_detector import GestureDetector
from openai_client import get_ai_response
from PIL import ImageFont, ImageDraw, Image
import numpy as np
import os

app = Flask(__name__)
cap = cv2.VideoCapture(0)
detector = GestureDetector()

latest_fingers = 0
latest_response = ""

# 한글 텍스트를 프레임에 표시하는 함수
def draw_korean_text(frame, text, position=(10, 30), font_size=24, font_path=None):
    if font_path is None:
        font_path = "C:/Windows/Fonts/malgun.ttf"  # Windows 기준
        # Linux일 경우 예: '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'

    image_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(image_pil)
    font = ImageFont.truetype(font_path, font_size)
    draw.text(position, text, font=font, fill=(255, 255, 255, 255))
    return cv2.cvtColor(np.array(image_pil), cv2.COLOR_RGB2BGR)

def gen_frames():
    global latest_fingers, latest_response
    while True:
        success, frame = cap.read()
        if not success:
            break

        fingers = detector.count_fingers(frame)
        if fingers != latest_fingers and fingers > 0:
            latest_fingers = fingers
            latest_response = get_ai_response(fingers)

        # 한글 텍스트 표시
        frame = draw_korean_text(frame, f'손가락 수: {latest_fingers}', (10, 30), font_size=28)
        frame = draw_korean_text(frame, latest_response, (10, 70), font_size=24)

        _, buffer = cv2.imencode('.jpg', frame)
        frame = buffer.tobytes()
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

@app.route('/')
def index():
    return render_template('index2.html')

@app.route('/video_feed')
def video_feed():
    return Response(gen_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

from flask import jsonify

@app.route('/get_latest_info')
def get_latest_info():
    return jsonify({
        "fingers": latest_fingers,
        "response": latest_response
    })

if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [23/Apr/2025 12:01:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2025 12:01:15] "GET /static/js/script.js HTTP/1.1" 304 -
127.0.0.1 - - [23/Apr/2025 12:01:15] "GET /video_feed HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2025 12:01:23] "GET /get_latest_info HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2025 12:01:25] "GET /get_latest_info HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2025 12:01:25] "GET /get_latest_info HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2025 12:01:25] "GET /get_latest_info HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2025 12:01:26] "GET /get_latest_info HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2025 12:01:26] "GET /get_latest_info HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2025 12:01:26] "GET /get_latest_info HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2025 12:01:26] "GET /get_latest_info HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2025 12:01:27] "GET /get_latest_info HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2025 12:01:35] "GET /get_latest_info 